## Linear Forecasting Project - Data Gathering


### Data Description

The variables used to forecast S&P500 index returns are listed as below:

1.Dividend-Price Ratio (DP) 
   
    -log of a twelve-month moving sum of dividends paid on the S&P 500 index minus the log of S&P 500 prices
    -source: http://www.multpl.com/

2.Price-to-Earnings Ratio (PE)

    -the price divided by earnings over the last 12 months
    -source: http://www.multpl.com/
    
3.Book-to-Market Ratio (BM)

    -book value of the S&P 500 divided by the S&P 500 index
    -source: Bloomberg

4.Cyclically Adjusted Price to Earnings Ratio (CAPE)

    -price divided by the average inflation adjusted earnings over the last ten years    
    -source: http://www.multpl.com/

5.Principal Component of Price Ratios (PCAprice)

    -the largest principal component of these variables  
    -source:calculated in calculateFirstPC.ipynb

6.Bond Yield (BY)

    -the 10-year Treasury bond yield divided by the bond yield EMA    
    -source:Quandl

7.Default Spread (DEF)

    -the difference between Baa yield and Aaa yield    
    -source:Quandl

8.Term Spread (TERM)

    -the yield difference between the 10-year Treasury Note and the three-month Treasury Bill    
    -source:Quandl

9.Cointegrating Residual of Consumption, Assets, and Wealth (CAY)

10.Sell in May and Go Away (SIM)

    -SIM = d/130, in which d is the number of days in the next 130 business days that lie between the second business day in May and the 15th business day of October  
    -source:calculated in calculateSIM.ipynb

11.Variance Risk Premium (VRP)

12.Implied Correlation (IC)

13.Baltic Dry Index (BDI)

    -source: Bloomberg

14.New Orders/Shipments (NOS)

15.Consumer Price Index (CPI)

    -the change in CPI over the last twelve months  
    -source: Quandl

16.Ratio of Stock Price to Commodity Price (PCR)

    -log of the ratio between SPY and GSCI  
    -Source: Bloomberg

17.Moving Average (MA)

    -0/1 signal based on SMA(20)-SMA(200)

18.Principal Component of Technical Indicators (PCAtech)

    -the first principal component of a set of technical indicators(Neely et al,2014)  
    -source:calculated in calculateFirstPC.ipynb

19.Oil Price Shocks (OIL)

    -the log of the current front oil futures price (CL1) minus the log of the fourth futures price (CL4) with a three month lag    
    -source: Bloomberg

20.Short Interest (SI)

    -the sum of all shares short on the NYSE divided by the average daily trading volume over the past 30 days    
    -source: Quandl


All the data are saved respectively in .csv format in datasets\hull, with the file name the same as the variable abbreviation.

In [1]:
import numpy as np
import pandas as pd
import datetime

In [10]:
# construct DataFrame for all raw data

path = r'linear forecasting\data\\'
#path = 'https://github.com/kailil/Linear_Forecasting/tree/master/Data/'

dailyVarNames = ['BY','DEF','TERM','VRP','IC','BDI','PCR','MA','PCAtech','OIL','SI','SIM'] # daily data
monthlyVarNames = ['CAY','NOS','BM','PE','CAPE','DP','PCAprice','CPI','SPX'] # monthly/quarterly data - need to fill this set to daily, SPX as date benchmark
varNames = dailyVarNames + monthlyVarNames
dateDict = {} # store the available time range

df_daily = pd.DataFrame()
for varName in dailyVarNames:
    df = pd.read_csv(path + varName+'.csv', index_col=0, parse_dates=[0], usecols=['Date',varName]) # montly at end of month
    df_daily = pd.concat([df_daily,df],axis=1)
    print(varName+ ' load successfully.')
    df = df.dropna(how='any')
    startDate = min(df.index)
    endDate = max(df.index)
    dateDict.update({varName: (startDate,endDate)})
                                                                                   
                                                                                   
df_monthly = pd.DataFrame()    
for varName in monthlyVarNames:
    df = pd.read_csv(path + varName+'.csv', index_col=0, parse_dates=[0], usecols=['Date',varName]) # montly at end of month
    df_monthly = pd.concat([df_monthly,df],axis=1)
    print(varName+ ' load successfully.')
    df = df.dropna(how='any')
    startDate = min(df.index)
    endDate = max(df.index)
    dateDict.update({varName: (startDate,endDate)})
                          

BY load successfully.
DEF load successfully.
TERM load successfully.
VRP load successfully.
IC load successfully.
BDI load successfully.
PCR load successfully.
MA load successfully.
PCAtech load successfully.
OIL load successfully.
SI load successfully.
SIM load successfully.
CAY load successfully.
NOS load successfully.
BM load successfully.
PE load successfully.
CAPE load successfully.
DP load successfully.
PCAprice load successfully.
CPI load successfully.
SPX load successfully.


D:\Anaconda2\envs\py360\lib\site-packages\pandas\core\indexes\base.py:2184: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


In [11]:
# pre-processing period
preStartDate = '1990-01-01'
preEndDate = '2017-06-30'


def selectRange(df, startDate,endDate):
    df_selected = df[(df.index >= datetime.datetime.strptime(startDate, '%Y-%m-%d')) 
                           & (df.index <= datetime.datetime.strptime(endDate, '%Y-%m-%d'))]
    return df_selected 
    
    
df_daily = selectRange(df_daily, preStartDate,preEndDate)
df_monthly = selectRange(df_monthly, preStartDate,preEndDate)

#df_daily = df_daily[(df_daily.index >= datetime.datetime.strptime(preStartDate, '%Y-%m-%d')) 
#                           & (df_daily.index <= datetime.datetime.strptime(preEndDate, '%Y-%m-%d'))]
#df_monthly = df_monthly[(df_monthly.index >= datetime.datetime.strptime(preStartDate, '%Y-%m-%d')) 
#                           & (df_monthly.index <= datetime.datetime.strptime(preEndDate, '%Y-%m-%d'))]

In [12]:
# fill the monthly data to daily data
# method: use the previous available data to fill the following missing ones
df_monthly = df_monthly.sort_index() # make sure the data is sorted ascendingly by dates
df_monthly.iloc[:,0:-1] = df_monthly.iloc[:,0:-1].fillna(method='pad') 
df_monthly = df_monthly.dropna(axis=0,how='any')

In [13]:
df = pd.concat([df_monthly,df_daily],axis=1,join='inner')
df = df.fillna(method='pad')
df
#df = df.drop(['SPX'],axis=1)

,CAY,NOS,BM,PE,CAPE,DP,PCAprice,CPI,SPX,BY,...,TERM,VRP,IC,BDI,PCR,MA,PCAtech,OIL,SI,SIM
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-31,0.022980,0.0275,0.506073,15.13,17.05,0.0328,-2.778152,0.051980,329.08,0.999436,...,0.43,NaN,NaN,0.001828,NaN,1.0,-0.987494,0.070086,NaN,0.515385
1990-02-01,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,328.79,0.999895,...,0.40,NaN,NaN,-0.003036,NaN,1.0,-0.388430,0.074084,NaN,0.523077
1990-02-02,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,330.92,1.000559,...,0.43,NaN,NaN,-0.005464,NaN,0.0,1.355371,0.076998,NaN,0.530769
1990-02-05,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,331.85,1.000244,...,0.45,NaN,NaN,-0.019382,NaN,0.0,2.567111,0.061805,NaN,0.538462
1990-02-06,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,329.66,1.000295,...,0.48,NaN,NaN,-0.028433,NaN,0.0,2.019740,0.065022,NaN,0.546154
1990-02-07,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,333.75,0.999666,...,0.44,NaN,NaN,-0.039903,NaN,0.0,3.042453,0.058214,NaN,0.553846
1990-02-08,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,332.96,0.999768,...,0.41,NaN,NaN,-0.042926,NaN,0.0,3.042453,0.055053,NaN,0.561538
1990-02-09,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,333.62,0.998688,...,0.29,NaN,NaN,-0.044659,NaN,0.0,3.042453,0.048117,NaN,0.569231
1990-02-12,0.022980,-0.0039,0.506073,14.97,16.51,0.0328,-2.778152,0.052632,330.08,1.000565,...,0.41,NaN,NaN,-0.045345,NaN,0.0,2.464329,0.055437,NaN,0.576923


In [19]:
# verstamp = str(datetime.datetime.now())[:16]
df.to_csv('linear forecasting\data\Predictors.csv')

In [20]:
# calculate SPX future returns

In [21]:
def getFutureReturn(df,ndays,colname):
    df = df.sort_index()
    r = df.pct_change(ndays).shift(-ndays)
    r.columns = [colname]
    return r


spx = pd.DataFrame(df['SPX'])

r1 = getFutureReturn(spx,22,'R_1M')
r2 = getFutureReturn(spx,65,'R_3M')
r3 = getFutureReturn(spx,126,'R_6M')
r4 = getFutureReturn(spx,252,'R_12M')

futRet = pd.concat([r1,r2,r3,r4],axis=1)
futRet = selectRange(futRet, preStartDate,preEndDate)
futRet.to_csv('linear forecasting\data\FutureReturn.csv')